In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jiyaparmar04","key":"f0c150fcf6b3e85cc011ec252d6a2cf6"}'}

In [2]:
# Install the Kaggle library
!pip install -q kaggle

# Set up the Kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

# Unzip the dataset
!unzip -q fashion-product-images-small.zip -d fashion-dataset

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): MIT
 96% 544M/565M [00:01<00:00, 320MB/s]
100% 565M/565M [00:01<00:00, 328MB/s]


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import os
import time

#Custom Dataset for the Kaggle Dataset
class KaggleFashionDataset(Dataset):
    def __init__(self, root_dir, annotations_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        # Load annotations and drop rows with missing images
        self.annotations = pd.read_csv(annotations_file, on_bad_lines='skip')
        self.annotations['image_path'] = self.annotations.apply(lambda row: os.path.join(self.root_dir, 'images', str(row['id']) + '.jpg'), axis=1)
        self.annotations = self.annotations[self.annotations['image_path'].apply(os.path.exists)]

        # Create a mapping from category name to a unique index
        self.classes = self.annotations['subCategory'].unique().tolist()
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')

        label_name = self.annotations.iloc[idx]['subCategory']
        label = self.class_to_idx[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label

# 2. Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 3. Data Loading
data_dir = 'fashion-dataset'
annotations_file = os.path.join(data_dir, 'styles.csv')
dataset = KaggleFashionDataset(root_dir=data_dir, annotations_file=annotations_file, transform=data_transforms)

dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)
class_names = dataset.classes
print(f"Training on {len(dataset)} images.")
print(f"Total classes: {len(class_names)}")

#  4. Model Fine-Tuning
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#  5. Training Loop
def train_model(model, criterion, optimizer, num_epochs=5):
    since = time.time()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    torch.save(model.state_dict(), 'kaggle_fashion_model.pth')
    print("Model saved to kaggle_fashion_model.pth")

if __name__ == '__main__':
    train_model(model, criterion, optimizer, num_epochs=5)


Using device: cuda:0
Training on 44419 images.
Total classes: 45


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 62.7MB/s]


Epoch 1/5
----------
Train Loss: 0.5512 Acc: 0.8710
Epoch 2/5
----------
Train Loss: 0.2186 Acc: 0.9451
Epoch 3/5
----------
Train Loss: 0.1655 Acc: 0.9579
Epoch 4/5
----------
Train Loss: 0.1314 Acc: 0.9659
Epoch 5/5
----------
Train Loss: 0.1127 Acc: 0.9703

Training complete in 12m 22s
Model saved to kaggle_fashion_model.pth


In [12]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!cp kaggle_fashion_model.pth "/content/drive/MyDrive/"


In [8]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import os

# 1. Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#  2. Get Class Names from the Dataset
if not os.path.exists('fashion-dataset'):
    print("Dataset not found. Unzipping from Google Drive...")
    pass

try:
    annotations_file = 'fashion-dataset/styles.csv'
    df = pd.read_csv(annotations_file, on_bad_lines='skip')
    class_names = df['subCategory'].unique().tolist()
    print(f"Found {len(class_names)} classes.")
except FileNotFoundError:
    print("Error: 'fashion-dataset/styles.csv' not found. Please re-run the Kaggle download steps.")
    class_names = []

#  3. Define Model Architecture
# This must be the same architecture as the one used for training.
model = models.resnet18(weights=None)
num_ftrs = model.fc.in_features
if class_names:
    model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

#  4. Load the Trained Weights
# Load the state_dict from your Google Drive
model_path = '/content/drive/MyDrive/kaggle_fashion_model.pth'
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print("Model weights loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}. Make sure it was saved correctly.")

#  5. Define Prediction Transforms
# These must be the same as the training transforms.
prediction_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#  6. Prediction Function
def predict_image(image_path, model, transform, class_list):
    if not class_list or model.fc is None:
        return "Model or class list not loaded correctly."

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        _, predicted_idx = torch.max(outputs, 1)

    predicted_class = class_list[predicted_idx.item()]
    confidence = probabilities[0][predicted_idx.item()].item()

    return predicted_class, confidence



Using device: cuda:0
Found 45 classes.
Model weights loaded successfully.


In [10]:

from google.colab import files

# Upload your image
uploaded = files.upload()

# Get the filename of the uploaded image
if uploaded:
    image_filename = next(iter(uploaded))
    print(f"\nUploaded '{image_filename}'")

    # Make a prediction
    predicted_category, confidence = predict_image(image_filename, model, prediction_transform, class_names)

    print(f"\nPredicted Category: {predicted_category}")
    print(f"Confidence: {confidence*100:.2f}%")
else:
    print("No file uploaded.")


Saving Image_9.jpg to Image_9.jpg

Uploaded 'Image_9.jpg'

Predicted Category: Shoes
Confidence: 99.63%


In [16]:
# Stage your files
!git add .

# Commit your work
!git commit -m "Your commit message"

# Push to GitHub
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
